## Workshop The Busy Engineer's Document Bucket

Bienvenido al taller del cubo de documentos para ingenieros ocupados. En este taller, se le guiará a través de la adición de cifrado del lado del cliente con KMS y AWS Encryption SDK. Cada ejercicio tiene instrucciones paso a paso. Cada ejercicio también tiene una -versión completa disponible para usar como referencia.

El sistema de ejemplo en el que trabajará es Document Bucket. Al comenzar este taller, este sistema utiliza DynamoDB y S3 para almacenar, buscar, enumerar y recuperar documentos con metadatos asociados. Los cuerpos de cada documento se escriben como objetos S3. DynamoDB mantiene un puntero a ese objeto, junto con los metadatos asociados para ese objeto. Además, DynamoDB está configurado para mantener registros de las etiquetas utilizadas para identificar el documento, de modo que pueda buscar documentos que coincidan con una etiqueta concreta.

Una vez completado su trabajo en el taller, habrá añadido cifrado del lado del cliente con AWS Encryption SDK y AWS KMS al Bucket de documentos. Antes de que los datos abandonen la instancia de origen en el almacén, se cifrarán en el lado del cliente. Una vez recuperados los datos, se descifrarán del lado del cliente con una comprobación de permisos y una entrada de auditoría de AWS KMS. También explorará opciones para vincular y validar los metadatos al objeto.

### Cómo empezar
#### Detalles del taller

En este taller, añadirá características de cifrado y descifrado al Cubo de documentos del ingeniero ocupado para conocer algunos patrones de AWS del mundo real para integrar el cifrado del lado del cliente mediante AWS Key Management Service (AWS KMS) y el SDK de cifrado de AWS (ESDK) en el código de la aplicación. Aprenderá a aprovechar características como la compatibilidad con varias claves de AWS KMS y el contexto de cifrado para proteger su aplicación.

Para comenzar, Document Bucket permite almacenar objetos (documentos o archivos u otros bloques de datos) en un bucket privado de Amazon S3 e indexarlos en Amazon DynamoDB. Esto permite a los usuarios de Document Bucket compartir objetos con otros usuarios y almacenarlos para su posterior recuperación. Las entradas de DynamoDB proporcionan un contexto de metadatos para cada objeto de Document Bucket, así como búsquedas rápidas en el contexto proporcionado.

Este contexto permite a los usuarios almacenar información adicional sobre el objeto S3, como el usuario de origen, la flota de destino, el proyecto o cualquier otra etiqueta que sería útil conocer sin tener que descargar y examinar el objeto.

DynamoDB está configurado para permitirle indexar en las claves asociadas a los objetos de S3. Así, por ejemplo, es una consulta rápida averiguar qué objetos se han etiquetado con un contexto de metadatos que incluya "configuración".

El s3 de documentos admite las siguientes API:

* **list**: Esta operación consulta en DynamoDB todas las entradas de todos los objetos del Cubo de Documentos, así como sus metadatos. Devuelve el conjunto de elementos que se han almacenado.

* **almacenar**: Esta operación acepta un blob de bytes y un mapa de contexto de metadatos. Genera un identificador único (UID) para el objeto. El UID y los metadatos asociados se escriben en DynamoDB. Los bytes del objeto se escriben en S3 bajo una clave de ese UID. Cualquier clave de metadatos de contexto en DynamoDB se actualiza para incluir ese nuevo UID.

* **Recuperar**: Esta operación acepta un identificador único (UID) como argumento. Busca el UID en DynamoDB para identificar el contexto asociado y, a continuación, recupera el objeto referenciado de S3. Devuelve el conjunto de documentos (UID, contexto y objeto de referencia) a la persona que realiza la llamada.

* **búsqueda**: Esta operación acepta una clave de metadatos como entrada de búsqueda. Busca en DynamoDB el conjunto de objetos del Cubo de documentos con un contexto que coincida con dicha clave. Devuelve el conjunto de UID y sus metadatos asociados.

* Una vez identificados los objetos deseados, puede pasar los UID a recuperar para obtener los objetos.
Esto es un comienzo para compartir, almacenar y buscar diversos objetos.

Pero, ¿qué ocurre con los documentos confidenciales? ¿O la protección, por ejemplo, de archivos de configuración importantes contra la corrupción accidental durante el almacenamiento o la recuperación?

### ¿Qué es el SDK de cifrado de AWS?

¿Qué es el AWS Encryption SDK?

El AWS Encryption SDK es una biblioteca de cifrado del lado del cliente diseñada para facilitar a todo el mundo el cifrado y descifrado de datos mediante estándares y prácticas recomendadas del sector. Le permite centrarse en la funcionalidad principal de su aplicación, en lugar de en cómo cifrar y descifrar mejor sus datos. El SDK de cifrado de AWS se proporciona de forma gratuita bajo la licencia Apache 2.0.

Para instalar:

>> pip install aws-encryption-sdk-cli

>> aws-encryption-cli --version

Ahora su Bucket de documentos cifrará los archivos cuando los almacene y los descifrará cuando los recupere. 

* En el archivo `kms-arn` se especifica el ARN de la clave de KMS que se utilizará para cifrar y descifrar los datos. 

* En el archivo `kms-context` se especifica el contexto de cifrado que se utilizará para cifrar y descifrar los datos. AWS KMS le permite especificar un contexto de cifrado en kms:Encrypt. Si lo hace, debe proporcionar exactamente el mismo contexto de cifrado en kms:Decrypt, o la operación fallará. (La coincidencia distingue entre mayúsculas y minúsculas, y los pares clave-valor se comparan de forma independiente del orden).

    Esta función protege contra los riesgos de que los textos cifrados sean manipulados, modificados o sustituidos, intencionadamente o no. Defiende tanto contra un atacante que sustituye un texto cifrado por otro como contra problemas como eventos operativos.



### Varias claves KMS.

Tal vez quieras que el Cubo de Documentos tenga dos regiones independientes para acceder a los contenidos, para una alta disponibilidad, o para poner los contenidos más cerca de los destinatarios.

Configurar varias claves KMS de este modo no requiere volver a cifrar los datos de los documentos. Esto se debe a que los datos siguen estando cifrados con una única clave de datos, utilizada exclusivamente para ese documento. La configuración de varias claves KMS hace que el SDK de AWS Encryption vuelva a cifrar esa clave de datos utilizando las claves KMS adicionales y almacene esa versión adicional de la clave de datos en el formato de mensaje cifrado. Mientras haya una clave KMS disponible para descifrar cualquier versión cifrada de la clave de datos, se podrá acceder al documento.

### Contexto de cifrado

Cuando se encriptan los datos se crea una carpeta con un archivo que tiene metadatos, denominados contexto, asociados a cada documento. Estos metadatos son un conjunto de pares de cadenas clave-valor.

Una de las características que proporcionan AWS KMS y AWS Encryption SDK se denomina contexto de cifrado. En esencia, el contexto de cifrado es este mapeo de metadatos: claves que se asocian con el contexto sobre el objeto y valores que indican información sobre cuál es ese contexto. Toda la información en el mapa no es secreta, y es la base para varias integraciones de características.

Un modelo útil para pensar en el contexto de encriptación es como afirmaciones sobre las Cinco Ws: Quién, Qué, Dónde, Cuándo, Por Qué. Por ejemplo:

* ¿Quién debería tener acceso a estos datos descifrados?
* ¿Qué datos se están descifrando?
* ¿Dónde se descifran?
* ¿Cuándo se utilizan estos datos?
* ¿Por qué se descifran estos datos?